Привет!
Тут немного не аккуратный код, просим прощения за сложность в читаемости.
Было бы не совсем правильно его менять после хакатона, поэтому это сообщение - единственное добавление в ноутбук. Решили его залить, чтобы проще было посмотреть что мы делали и как.

# Докинг отобранных молекул

Установка и импорт библиотек, а также разрешение на работу с диском (нужно разрешить)

In [ ]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 38.7 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Список названий датасетов, которые нужно объединить (указывай имена файлов без путей)
datasets_to_combine = [
    'checkpoint_iter_5.csv',
    'top_molecules_final.csv',
    'top_molecules_final_1.csv',
    'top_molecules_final_2.csv'
]

# Список для хранения датафреймов
dataframes = []

# Читаем каждый указанный датасет и добавляем метку модели
for dataset in datasets_to_combine:
    # Извлекаем имя модели из названия файла (первая часть до '_')
    model_name = dataset.split('_')[0]
    df = pd.read_csv(dataset)
    df['Model'] = model_name
    dataframes.append(df)

# Объединяем все датафреймы
if dataframes:
    df_combined = pd.concat(dataframes, ignore_index=True)

    # Фильтрация по уникальным SMILES (оставляем первую запись)
    df_combined = df_combined.drop_duplicates(subset=['SMILES'], keep='first')

    # Сохраняем объединенный датасет
    output_file = 'combined_dataset.csv'
    df_combined.to_csv(output_file, index=False)
    print(f"Объединенный датасет сохранен как: {output_file}")
    print(f"Удалено {len(dataframes[0]) + len(dataframes[1]) - len(df_combined)} дубликатов по SMILES.")
else:
    print("Указанные датасеты не найдены или список пуст.")

Объединенный датасет сохранен как: combined_dataset.csv
Удалено -105 дубликатов по SMILES.


Формируем 3д структуры конформеров по смайлсам из подгруженного файла. Они сохраняются в папку  '/content/drive/My Drive/gnina/target/CYP3a4_ic50_df'

In [ ]:
import os
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors

output_dir = '/content/drive/My Drive/gnina/target/modelRNN'
os.makedirs(output_dir, exist_ok=True)

df = pd.read_csv('combined_dataset.csv')

for i, smi in enumerate(df['SMILES'], start=1):
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        print(f"Молекула с SMILES {smi} не распознана")
        continue

    # Calculate molecular weight
    mol_weight = Descriptors.MolWt(mol)
    if mol_weight > 500000:
        print(f"Молекула {i} с SMILES {smi} исключена (молекулярная масса {mol_weight:.2f} > 500)")
        continue

    mol = Chem.AddHs(mol)

    # Генерация 3D конформера
    embed_status = AllChem.EmbedMolecule(mol, AllChem.ETKDG())
    if embed_status != 0:
        print(f"Не удалось сгенерировать 3D конформер для молекулы {i} со SMILES {smi}")
        continue

    # Оптимизация структуры
    try:
        AllChem.MMFFOptimizeMolecule(mol)
    except ValueError as e:
        print(f"Ошибка оптимизации молекулы {i}: {e}")
        continue

    pdb_filename = os.path.join(output_dir, f"ligand_{i}.pdb")
    with open(pdb_filename, 'w') as f:
        f.write(Chem.MolToPDBBlock(mol))
    print(f"Сохранён файл {pdb_filename}")

Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_1.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_2.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_3.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_4.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_5.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_6.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_7.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_8.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_9.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_10.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_11.pdb
Не удалось сгенерировать 3D конформер для молекулы 12 со SMILES BrC1=C(NC(C(C)C)=O)C2=CC(=C1)OCO2
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_13.pdb
Сохранён файл /content/

[19:22:48] UFFTYPER: Unrecognized atom type: S_5+4 (5)


Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_61.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_62.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_63.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_64.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_65.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_66.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_67.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_68.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_69.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_70.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_71.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_72.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_73.pdb
Сохранён файл /content/drive/My Drive/

[19:22:51] UFFTYPER: Unrecognized atom type: S_5+4 (1)


Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_148.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_149.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_150.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_151.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_152.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_153.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_154.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_155.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_156.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_157.pdb


...

Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_164.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_165.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_166.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_167.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_168.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_169.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_170.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_171.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_172.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_173.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_174.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_175.pdb


[19:22:52] UFFTYPER: Warning: hybridization set to SP3 for atom 0
[19:22:52] UFFTYPER: Warning: hybridization set to SP3 for atom 2


Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_176.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_177.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_178.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_179.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_180.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_181.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_182.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_183.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_184.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_185.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_186.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_187.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_188.pdb
Сохранён файл /content/dr

[19:22:54] UFFTYPER: Unrecognized atom type: S_5+4 (14)


Не удалось сгенерировать 3D конформер для молекулы 211 со SMILES C1CCCC#C1
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_212.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_213.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_214.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_215.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_216.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_217.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_218.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_219.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_220.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_221.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_222.pdb
Сохранён файл /content/drive/My Drive/gnina/target/modelRNN/ligand_223.pdb
Сохранён файл /content/dr

После следующей команды должна вылезти в рамочке красивой информация о видеокарте. Если нет - нужно подключить GPU (Runtme->Change runtime type->Hardware Accelerator=GPU) - и тогда вылезет.

Скачиваем гнину! Займёт какое-то время.

In [ ]:
!wget https://github.com/gnina/gnina/releases/download/v1.3/gnina

--2025-07-16 01:11:54--  https://github.com/gnina/gnina/releases/download/v1.3/gnina
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/45548146/f37c6a31-c8d1-4c4f-9748-c7c6f727e868?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-07-16T01%3A47%3A04Z&rscd=attachment%3B+filename%3Dgnina&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-07-16T00%3A46%3A38Z&ske=2025-07-16T01%3A47%3A04Z&sks=b&skv=2018-11-09&sig=XUV875B0y11TMbVVa1fMqYd3XrYaR9SlfDgZkG19mXQ%3D&jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1MjYyODYxNCwibmJmIjoxNzUyNjI4MzE0LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5ibG9iLmNvcmUud2luZG93cy5uZXQifQ.j4

In [ ]:
!chmod +x gnina

In [ ]:
!./gnina --version

gnina v1.3 master:97fa6bc+   Built Oct  3 2024.


Выбираем только атомы белка для pdb файла с которым будем работать.

In [ ]:
!grep ATOM  4L7B_protein.pdb > rec.pdb

Найдём координаты кармана связывания

In [ ]:
import sys

def parse_pdb(pdb_file, chain_id, residue_num=None, residue_name=None):
    center_x, center_y, center_z = 0.0, 0.0, 0.0
    min_x, max_x, min_y, max_y, min_z, max_z = float('inf'), float('-inf'), float('inf'), float('-inf'), float('inf'), float('-inf')
    atom_count = 0

    with open(pdb_file, 'r') as f:
        for line in f:
            if line.startswith('HETATM'):
                current_chain = line[21:22].strip()
                current_resnum = line[22:26].strip()
                current_resname = line[17:20].strip()
                if (current_chain == chain_id and
                    (residue_num is None or current_resnum == residue_num) and
                    (residue_name is None or current_resname == residue_name)):
                    x = float(line[30:38])
                    y = float(line[38:46])
                    z = float(line[46:54])
                    print(f"Обнаружен атом: {line.strip()}")  # Отладочный вывод
                    center_x += x
                    center_y += y
                    center_z += z
                    atom_count += 1
                    min_x = min(min_x, x)
                    max_x = max(max_x, x)
                    min_y = min(min_y, y)
                    max_y = max(max_y, y)
                    min_z = min(min_z, z)
                    max_z = max(max_z, z)

    if atom_count == 0:
        raise ValueError(f"Атомы лиганда с цепью {chain_id} и параметрами (resnum={residue_num}, resname={residue_name}) не найдены.")

    center_x /= atom_count
    center_y /= atom_count
    center_z /= atom_count
    size_x = max_x - min_x + 10
    size_y = max_y - min_y + 10
    size_z = max_z - min_z + 10

    return center_x, center_y, center_z, size_x, size_y, size_z

if __name__ == "__main__":
    pdb_file = "4L7B.pdb"  # Имя загруженного файла
    chain_id = "B"         # Цепь
    residue_num = "701"    # Номер остатка
    residue_name = "1VV"   # Имя остатка

    try:
        center_x, center_y, center_z, size_x, size_y, size_z = parse_pdb(pdb_file, chain_id, residue_num, residue_name)
        print(f"--center_x {center_x:.3f}")
        print(f"--center_y {center_y:.3f}")
        print(f"--center_z {center_z:.3f}")
        print(f"--size_x {size_x:.3f}")
        print(f"--size_y {size_y:.3f}")
        print(f"--size_z {size_z:.3f}")
    except ValueError as e:
        print(f"Ошибка: {e}")
    except Exception as e:
        print(f"Произошла ошибка: {e}")

Произошла ошибка: [Errno 2] No such file or directory: '4L7B.pdb'


Докинг!

In [ ]:
import os
import subprocess
from google.colab import drive

# Монтируем Google Drive
drive.mount('/content/drive')

# Пути
source_dir = '/content/drive/My Drive/gnina/target/RealMolecule'  # Откуда берём лиганды
work_dir = "/content/"  # Рабочая директория
results_dir = "/content/drive/MyDrive/gnina/datacon_res/RealMolecule"  # Куда сохраняем результаты
os.makedirs(results_dir, exist_ok=True)

# # Копируем rec.pdb в рабочую директорию
# rec_source = os.path.join(source_dir, "rec.pdb")
# if os.path.exists(rec_source):
#     !cp "{rec_source}" "{work_dir}rec.pdb"
# else:
#     print(f"Файл {rec_source} не найден!")

# Проходим по всем файлам лигандов
for ligand_file in os.listdir(source_dir):
    if ligand_file.endswith(".pdb") and ligand_file != "rec.pdb":
        ligand_name = os.path.splitext(ligand_file)[0]  # Имя без расширения, например "ligand_27"

        # Копируем лиганд в рабочую директорию
        ligand_source = os.path.join(source_dir, ligand_file)
        !cp "{ligand_source}" "{work_dir}{ligand_file}"

        # Формируем команду для GNINA
        output_sdf = os.path.join(results_dir, f"docked_{ligand_name}.sdf")
        log_file = os.path.join(results_dir, f"{ligand_name}_log.txt")
        command = (
            f"./gnina -r rec.pdb -l {ligand_file} "
            f"--center_x -3.561 --center_y 2.506 --center_z -27.501 "
            f"--size_x 17.528 --size_y 18.079 --size_z 18.267 "
            f"-o {output_sdf} --seed 0 --cnn fast"
        ).strip()  # Удаляем лишние пробелы в конце

        print(f"Запуск докинга для {ligand_file}...")
        # Выполняем команду и сохраняем вывод в лог-файл
        with open(log_file, "w") as f:
            process = subprocess.run(command, shell=True, stdout=f, stderr=subprocess.STDOUT, text=True)

        if process.returncode == 0:
            print(f"Докинг для {ligand_file} завершён. Результаты в {output_sdf} и {log_file}")
        else:
            print(f"Ошибка при докинге для {ligand_file}. Подробности в {log_file}")
            with open(log_file, "r") as f:
                print(f"Содержимое лог-файла:\n{f.read()}")

# Очистка временных файлов (опционально)
!rm -f rec.pdb *.pdb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Запуск докинга для ligand_1.pdb...
Докинг для ligand_1.pdb завершён. Результаты в /content/drive/MyDrive/gnina/datacon_res/RealMolecule/docked_ligand_1.sdf и /content/drive/MyDrive/gnina/datacon_res/RealMolecule/ligand_1_log.txt
Запуск докинга для ligand_2.pdb...
Докинг для ligand_2.pdb завершён. Результаты в /content/drive/MyDrive/gnina/datacon_res/RealMolecule/docked_ligand_2.sdf и /content/drive/MyDrive/gnina/datacon_res/RealMolecule/ligand_2_log.txt
Запуск докинга для ligand_3.pdb...
Докинг для ligand_3.pdb завершён. Результаты в /content/drive/MyDrive/gnina/datacon_res/RealMolecule/docked_ligand_3.sdf и /content/drive/MyDrive/gnina/datacon_res/RealMolecule/ligand_3_log.txt


Извлечение афинности из логов GNINA и обновление датасета

In [ ]:
import os
import re
from rdkit import Chem
import pandas as pd

folder = "/content/drive/MyDrive/gnina/datacon_res/RealMolecule"

sdf_smiles = {}
log_affinities = {}

# Чтение SDF → SMILES
for fname in os.listdir(folder):
    if fname.endswith(".sdf") and fname.startswith("docked_ligand_"):
        mol_path = os.path.join(folder, fname)
        suppl = Chem.SDMolSupplier(mol_path)
        for mol in suppl:
            if mol is not None:
                smiles = Chem.MolToSmiles(mol)
                ligand_id = fname.replace("docked_", "").replace(".sdf", "")  # ligand_1
                sdf_smiles[ligand_id] = smiles
                break

# Чтение LOG → best affinity
for fname in os.listdir(folder):
    if fname.endswith("_log.txt") and fname.startswith("ligand_"):
        log_path = os.path.join(folder, fname)
        with open(log_path, 'r') as f:
            content = f.read()
        matches = re.findall(r'\n\s+\d+\s+([-+]?[0-9]*\.?[0-9]+)\s+', content)
        if matches:
            affinities = [float(a) for a in matches]
            best_affinity = min(affinities)
            ligand_id = fname.replace("_log.txt", "")  # ligand_1
            log_affinities[ligand_id] = best_affinity

# Собираем таблицу
rows = []
for ligand_id, smiles in sdf_smiles.items():
    affinity = log_affinities.get(ligand_id)
    if affinity is not None:
        rows.append({'SMILES': smiles, 'best_affinity': affinity})
    else:
        print(f"[WARN] no log for: {ligand_id}")

df = pd.DataFrame(rows)
print(df)

# Сохранить при желании
df.to_csv("ligand_affinities.csv", index=False)


                      SMILES  best_affinity
0  NC(/C=[SH]/CC(=O)O)C(=O)O          -5.19
1        COC(=O)/C=C/C(=O)OC          -4.54


[01:18:30] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[01:18:30] Explicit valence for atom # 31 N, 4, is greater than permitted
[01:18:30] ERROR: Could not sanitize molecule ending on line 109
[01:18:30] ERROR: Explicit valence for atom # 31 N, 4, is greater than permitted
[01:18:30] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[01:18:30] Explicit valence for atom # 31 N, 4, is greater than permitted
[01:18:30] ERROR: Could not sanitize molecule ending on line 258
[01:18:30] ERROR: Explicit valence for atom # 31 N, 4, is greater than permitted
[01:18:30] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
[01:18:30] Explicit valence for atom # 31 N, 4, is greater than permitted
[01:18:30] ERROR: Could not sanitize molecule ending on line 407
[01:18:30] ERROR: Explicit valence for atom # 31 N, 4, is greater than permitted
[01:

In [ ]:
import os
import re
import pandas as pd
from google.colab import drive

# Подключаем Google Drive
drive.mount('/content/drive')

# Папка с логами и датасетами
log_dir = '/content/drive/MyDrive/gnina/datacon_res/'
output_dir = '/content/drive/MyDrive/gnina/datacon_res/'
input_csv = 'combined_dataset.csv'  # Замени на путь к твоему CSV

# Читаем исходный датасет
df = pd.read_csv(input_csv)

# Функция для извлечения минимальной афинности из лога
def extract_affinity(log_file):
    with open(log_file, 'r') as f:
        content = f.read()
        # Ищем все значения affinity (kcal/mol) с помощью регулярного выражения
        affinities = re.findall(r'(\-?\d+\.\d+)\s+\-?\d+\.\d+\s+\d+\.\d+\s+\d+\.\d+', content)
        if affinities:
            return min(float(a) for a in affinities)  # Берем минимальное значение
        return None

# Обработка каждого лиганда
for i in range(len(df)):
    log_file = os.path.join(log_dir, f'ligand_{i}_log.txt')
    if os.path.exists(log_file):
        affinity = extract_affinity(log_file)
        if affinity is not None:
            df.loc[i, 'Docking_Affinity'] = affinity

# Сохраняем обновленный датасет
output_csv = os.path.join(output_dir, 'diffsbdd_dataset_with_docking.csv')
df.to_csv(output_csv, index=False)
print(f"Обновленный датасет сохранен в: {output_csv}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: 'combined_dataset.csv'

Сравнение датасетов и выявление лучших молекул

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive

# Подключаем Google Drive
drive.mount('/content/drive')

# Пути к датасетам
diffsbdd_csv = '/content/drive/MyDrive/gnina/datacon_res/diffsbdd/diffsbdd_dataset_with_docking.csv'
rl_csv = '/content/drive/MyDrive/gnina/datacon_res/rl/rl_dataset_with_docking.csv'  # Замени на путь к RL CSV

# Читаем датасеты
df_diffsbdd = pd.read_csv(diffsbdd_csv)
df_rl = pd.read_csv(rl_csv)

# Добавляем метку модели
df_diffsbdd['Model'] = 'DiffSBDD'
df_rl['Model'] = 'RL'

# Объединяем датасеты
df_combined = pd.concat([df_diffsbdd, df_rl], ignore_index=True)

# Список метрик для нормализации (добавляем Docking_Affinity)
metrics = ['QED', 'CNS_MPO', 'pIC50_KEAP1', 'pIC50_EGFR', 'pIC50_IKKb', 'Selectivity_Score', 'Score', 'Docking_Affinity']

# Функция нормализации с сигмоидой (для минимизации, например, Docking_Affinity, инвертируем)
def sigmoid_normalize(x, reverse=False):
    if reverse:
        x = -x  # Инверсия для минимизации (нижние значения лучше)
    return 1 / (1 + np.exp(-x))

# Нормализация метрик
for metric in metrics:
    if metric in df_combined.columns:
        if metric == 'Docking_Affinity':  # Минимизация (чем ниже, тем лучше)
            df_combined[f'Normalized_{metric}'] = sigmoid_normalize(df_combined[metric], reverse=True)
        else:  # Максимум (чем выше, тем лучше)
            df_combined[f'Normalized_{metric}'] = sigmoid_normalize(df_combined[metric])

# Вычисление итогового скора (среднее по нормализованным метрикам)
df_combined['Total_Score'] = df_combined[[f'Normalized_{m}' for m in metrics if f'Normalized_{m}' in df_combined.columns]].mean(axis=1)

# Выбираем топ-5 молекул
top_molecules = df_combined.sort_values(by='Total_Score', ascending=False).head(5)

# Сохраняем результаты
output_combined = '/content/drive/MyDrive/gnina/datacon_res/combined_dataset_with_scores.csv'
top_molecules.to_csv(output_combined, index=False)
print(f"Топ-5 молекул сохранены в: {output_combined}")
print("\nТоп-5 молекул:")
print(top_molecules[['SMILES', 'Model', 'Total_Score'] + [f'Normalized_{m}' for m in metrics]])

Визуализация результатов

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from google.colab import drive

# Подключаем Google Drive
drive.mount('/content/drive')

# Читаем объединенный датасет
combined_csv = '/content/drive/MyDrive/gnina/datacon_res/combined_dataset_with_scores.csv'
df = pd.read_csv(combined_csv)

# Настройка стиля
plt.style.use('seaborn')
sns.set_palette("husl")

# 1. График распределения Docking_Affinity по моделям
plt.figure(figsize=(10, 6))
sns.boxplot(x='Model', y='Docking_Affinity', data=df)
plt.title('Распределение афинности докинга по моделям')
plt.xlabel('Модель')
plt.ylabel('Афинность (kcal/mol)')
plt.savefig('/content/drive/MyDrive/gnina/datacon_res/docking_affinity_boxplot.png')
plt.close()

# 2. График корреляции между метриками
plt.figure(figsize=(12, 8))
sns.heatmap(df[['QED', 'CNS_MPO', 'pIC50_KEAP1', 'pIC50_EGFR', 'pIC50_IKKb', 'Selectivity_Score', 'Score', 'Docking_Affinity', 'Total_Score']].corr(), annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Корреляция между метриками')
plt.savefig('/content/drive/MyDrive/gnina/datacon_res/correlation_heatmap.png')
plt.close()

# 3. Визуализация топ-5 молекул
top_df = df.sort_values(by='Total_Score', ascending=False).head(5)
plt.figure(figsize=(12, 6))
bar_width = 0.15
index = np.arange(len(top_df))
metrics_to_plot = ['Normalized_pIC50_KEAP1', 'Normalized_Docking_Affinity', 'Normalized_Total_Score']

for i, metric in enumerate(metrics_to_plot):
    plt.bar(index + i * bar_width, top_df[metric], bar_width, label=metric.replace('Normalized_', ''))

plt.xlabel('Молекулы (Топ-5)')
plt.ylabel('Нормализованное значение')
plt.title('Сравнение характеристик топ-5 молекул')
plt.xticks(index + bar_width, [f"{i+1} ({m})" for i, m in enumerate(top_df['Model'])], rotation=45)
plt.legend()
plt.savefig('/content/drive/MyDrive/gnina/datacon_res/top_molecules_comparison.png')
plt.close()

# 4. График по моделям для топ-5
plt.figure(figsize=(10, 6))
sns.barplot(x='SMILES', y='Total_Score', hue='Model', data=top_df)
plt.xticks(rotation=90)
plt.title('Топ-5 молекул по общему скору по моделям')
plt.xlabel('SMILES')
plt.ylabel('Общий скор')
plt.savefig('/content/drive/MyDrive/gnina/datacon_res/top_molecules_by_model.png')
plt.close()

print("Графики сохранены в /content/drive/MyDrive/gnina/datacon_res/")